<h1 align = "center"> Home Value Predictions in Greater Boston Area </h1> 

<img src="static/img/mls_listing.png" width=800 align="center">

<h6 align = "center"> MLS Listing Preview </h6> 

## Introduction

A home is often the largest and most expensive purchase a person makes in his or her lifetime. Ensuring homeowners have a trusted way to monitor this asset is incredibly important. While many individual homebuyers are less sensitive to price forecasts, small errors in price prediction can have systemic negative effects in the economy as a whole. Accurate prediction makes it easier to understand which features would influence the final property price.

## Problem Statement

Real estate prices are very much dependent on factors that are not easy to control. Analyzing broader market conditions and specific property determinants in order to establish how property values may change over the course of time are utterly important. Massive data can be obtained about the current market situation, which demands using powerful machine learning algorithms in order to predict with high precision and in a reasonable time frame.

*Goal: Explore different models using internal data source (MLS data). Choose the best model as the final home valuation algorithms to predict prices on a new set of listings. Use price predictions and other listing features to build the app for the "Midterm_Part_II".*

### Load libraries

In [1]:
import pandas as pd
import numpy as np

### Load Data

Before we start with modelling, we have to take a closer look at our data features:

In [2]:
# Read dataset columns
all_columns = pd.read_csv('static/data/sold_geocoded.csv',low_memory=False,dtype='unicode').columns.values
all_columns

array(['Unnamed: 0', 'DisplayX', 'DisplayY', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'AGE', 'AGENTNAME', 'AREA', 'BATHS', 'BEDS', 'COOLING',
       'DOM', 'DTO', 'ELEMENTARYSCHOOL', 'EXPIREDDATE', 'GARAGE',
       'HEATING', 'HIGHSCHOOL', 'HOUSENUM1', 'HOUSENUM2',
       'JUNIORHIGHSCHOOL', 'LEVEL', 'LISTDATE', 'LISTPRICE', 'LOTSIZE',
       'MLSNUM', 'MONTH_ORIG', 'OFFICENAME', 'OFFICEPHONE',
       'OTHERFEATURES', 'PHOTOURL', 'PROPTYPE', 'REMARKS',
       'SHOWINGINSTRUCTIONS', 'SOLDDATE', 'SOLDPRICE', 'SQFT', 'STATUS',
       'STREETNAME', 'STYLE', 'TYPE', 'YEAR_ORIG', 'zoMLSNUM',
       'META_STATUS', 'MOS', 'Percent_diff', 'PPSF'], dtype=object)

Clearly, there are too many features in the dataset. We don't need all of them for this task. Let us choose a good set of predictors. 

In [3]:
use_cols = ['MLSNUM','DisplayX','DisplayY','ZIP','AGE','BATHS','BEDS','GARAGE','LISTDATE',
            'LISTPRICE','LOTSIZE','PROPTYPE','SOLDPRICE','SQFT', 'PPSF', 'PHOTOURL', 'REMARKS']

Data description:

    1. MLSNUM - listing number
    2. DisplayX - latitude
    3. DisplayY - longitude
    4. ZIP - ZIP code
    5. AGE - age of the building
    6. BATHS - number of bathrooms 
    7. BEDS - number of bedrooms 
    8. GARAGE - number of parking spots in the building
    9. LISTDATE - listed date of sale
    10. LISTPRICE - asking price
    11. LOTSIZE - building lotsize
    12. PROPTYPE - building type (CC-condo; MF-multifamily, SF-single family)
    13. SOLDPRICE - condo final price
    14. SQFT - total condo square feet
    15. PPSF - price per square foot
    16. PHOTOURL - single condo picture URL
    17. REMARKS - descriptions

In [4]:
# Load data with selected set of features
df_all = pd.read_csv('static/data/sold_geocoded.csv', usecols=use_cols)
df_all.head()

,DisplayX,DisplayY,ZIP,AGE,BATHS,BEDS,GARAGE,LISTDATE,LISTPRICE,LOTSIZE,MLSNUM,PHOTOURL,PROPTYPE,REMARKS,SOLDPRICE,SQFT,PPSF
0,-71.003531,42.834870,1860,12.0,1.0,2,2.0,3/26/2013 0:00,169900.0,NaN,71498924.0,http://media.mlspin.com/photo.aspx?mls=71498924,CC,Grange Hall Condominiums in downtown Merrimac....,177500.0,1283,138.347623
1,-71.893472,42.199845,1524,3.0,2.0,2,2.0,3/30/2013 0:00,279900.0,1306800.0,71500500.0,http://media.mlspin.com/photo.aspx?mls=71500500,CC,2”X6” WALL CONSTRUCTION-ENERGY EFFICIENT HOMES...,295850.0,1368,216.264620
2,-71.437349,42.653706,1879,5.0,2.5,2,2.0,10/10/2013 0:00,449900.0,5000.0,71595485.0,http://media.mlspin.com/photo.aspx?mls=71595485,CC,This 1.850 sq.ft. To be Built Pine Valley II s...,470519.0,1850,254.334595
3,-70.888071,42.507990,1970,117.0,2.0,2,1.0,10/21/2013 0:00,389500.0,8400.0,71599223.0,http://media.mlspin.com/photo.aspx?mls=71599223,CC,Salem - Water view Condo in renovated quality ...,383500.0,1502,255.326232
4,-70.904487,42.518712,1970,38.0,1.0,2,0.0,4/15/2014 0:00,237000.0,NaN,71661544.0,http://media.mlspin.com/photo.aspx?mls=71661544,CC,Welcome to this two bedroom townhouse located ...,232000.0,1469,157.930565


### Data Preprocessing

This section includes preprocessing, technique necessary for the raw data transformation into an understandable format. MLS data may be often incomplete, inconsistent, and/or lacking in certain infomration, and is likely to contain many errors. Data preprocessing is a proven method of resolving such issues.

In [5]:
# Clean PPSF data
# PPSF has many "inf" values that need to be removed
# Take a look at the function below and 
# make sure there are no any "nan" or "inf" values left

df_all['PPSF'] = df_all['PPSF'].round(2)

# array to store indices
indexes = []

# colect all indices that contain "inf" value
for p in enumerate(df_all['PPSF']):
    if p[1] == np.inf:
        indexes.append(p[0])
        
# drop all rows with "inf" values        
df_all = df_all.drop(df_all.index[indexes])  

In [6]:
# Check the shape of the data - you should be able to see (158874, 17)
df_all.shape


(158874, 17)

YOUR TURN <br><br>
**Question 1 (1 pt):** <br> *Create new column 'LISTMONTH', extracting the information from the 'LISTDATE' feature.*

In [7]:
# YOUR CODE HERE
list_month = []


for index, p in df_all.iterrows():
    list_month.append(p['LISTDATE'].split('/')[0])

df_all['LISTDATE'] = df_all['LISTDATE'].astype('datetime64[ns]')
df_all['LISTMONTH'] = list_month
df_all['LISTMONTH'] = df_all['LISTMONTH'].astype('int64')

In [8]:
# Check datatypes
df_all.dtypes

DisplayX            float64
DisplayY            float64
ZIP                   int64
AGE                 float64
BATHS               float64
BEDS                  int64
GARAGE              float64
LISTDATE     datetime64[ns]
LISTPRICE           float64
LOTSIZE             float64
MLSNUM              float64
PHOTOURL             object
PROPTYPE             object
REMARKS              object
SOLDPRICE           float64
SQFT                  int64
PPSF                float64
LISTMONTH             int64
dtype: object

The above output should be exactly the same as this one:<br>
<img src="static/img/dtypes.png" align="left">

YOUR TURN <br> <br> **Question 2 (1 pt):** <br> *One-hot encode 'ZIP' feature, using pandas get_dummies() method.*

In [9]:
# YOUR CODE HERE
df_all = pd.get_dummies(df_all,columns=['ZIP'])



In [10]:
# The final cleaning step is to take care of missing values
# Drop all "nan" rows/cols using .dropna() pandas method
df_all.dropna(inplace=True)

In [11]:
# Final check for "nan" values
# Notice the use of .head(18) here!!!
# We are preventing long outputs and
# try to print only columns that corespond to "use_cols" features set
# Important: Remove .head(18) to check all features

df_all.isnull().sum()


DisplayX     0
DisplayY     0
AGE          0
BATHS        0
BEDS         0
            ..
ZIP_20771    0
ZIP_20779    0
ZIP_92148    0
ZIP_92664    0
ZIP_92962    0
Length: 725, dtype: int64

YOUR TURN <br><br>
**Question 3 (1 pt):** <br> *Extract condos data and create the new dataframe called "condos".* 
<br><br>Hint: PROPTYPE has 3 unique values: 'CC','MF', and 'CC', where 'CC' stands for **condos**.

In [12]:
# YOUR CODE HERE
condo_list = []

for index, p in enumerate(df_all['PROPTYPE']):
    if p == 'CC':
        df = pd.Series(df_all.iloc[index])
        condo_list.append(df)
    
condos = pd.DataFrame(data=condo_list, columns=list(df_all.columns))
condos
# collect all indices that contain "CC" value


,DisplayX,DisplayY,AGE,BATHS,BEDS,GARAGE,LISTDATE,LISTPRICE,LOTSIZE,MLSNUM,...,ZIP_20184,ZIP_20302,ZIP_20382,ZIP_20452,ZIP_20743,ZIP_20771,ZIP_20779,ZIP_92148,ZIP_92664,ZIP_92962
1,-71.893472,42.199845,3.0,2.0,2,2.0,2013-03-30,279900.0,1306800.0,71500500.0,...,0,0,0,0,0,0,0,0,0,0
2,-71.437349,42.653706,5.0,2.5,2,2.0,2013-10-10,449900.0,5000.0,71595485.0,...,0,0,0,0,0,0,0,0,0,0
3,-70.888071,42.507990,117.0,2.0,2,1.0,2013-10-21,389500.0,8400.0,71599223.0,...,0,0,0,0,0,0,0,0,0,0
10,-71.625610,42.061016,0.0,1.0,2,2.0,2014-09-10,219990.0,7000.0,71741165.0,...,0,0,0,0,0,0,0,0,0,0
14,-71.275461,42.486940,3.0,1.5,2,0.0,2014-10-03,399000.0,45011.0,71752429.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154134,-70.905711,42.525080,113.0,1.0,2,0.0,2017-11-30,199900.0,5964.0,72260020.0,...,0,0,0,0,0,0,0,0,0,0
154135,-71.162691,42.346250,118.0,1.0,2,0.0,2017-12-01,499000.0,5500.0,72260387.0,...,0,0,0,0,0,0,0,0,0,0
154136,-71.300620,42.665132,33.0,1.5,2,1.0,2017-12-01,174900.0,0.0,72260416.0,...,0,0,0,0,0,0,0,0,0,0
154146,-71.732934,42.151581,1.0,2.5,2,2.0,2017-09-19,499900.0,0.0,72263694.0,...,0,0,0,0,0,0,0,0,0,0


Let us sort all condo values by 'LISTDATE' and check the final shape of the data:

In [13]:
# Sort all values by 'LISTDATE'
condos = condos.sort_values(by='LISTDATE')
condos.head()

,DisplayX,DisplayY,AGE,BATHS,BEDS,GARAGE,LISTDATE,LISTPRICE,LOTSIZE,MLSNUM,...,ZIP_20184,ZIP_20302,ZIP_20382,ZIP_20452,ZIP_20743,ZIP_20771,ZIP_20779,ZIP_92148,ZIP_92664,ZIP_92962
122119,-70.935221,42.497680,31.0,2.5,3,1.0,2012-05-06,295000.0,938718.0,71378749.0,...,0,0,0,0,0,0,0,0,0,0
23494,-71.435071,42.652326,2.0,2.5,2,2.0,2013-02-23,612799.0,4597.0,72109900.0,...,0,0,0,0,0,0,0,0,0,0
1,-71.893472,42.199845,3.0,2.0,2,2.0,2013-03-30,279900.0,1306800.0,71500500.0,...,0,0,0,0,0,0,0,0,0,0
113725,-71.316857,42.362081,3.0,3.5,3,2.0,2013-04-28,2795000.0,0.0,71515996.0,...,0,0,0,0,0,0,0,0,0,0
13698,-70.934101,41.543220,118.0,2.5,2,0.0,2013-06-21,379999.0,0.0,71545648.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Condos shape - you shoud see (10027, 725)
condos.shape

(10027, 725)

### Split data into "future_data" and "training_data"

So far we have worked on cleaning the data. Now, we want to split it into two datasets: "training_data" and "future_data". The first dataset ("training_data") as its name implies will serve for training and second dataset  ("future_data") will act like a future, "unseen" data. The final (best) model will be used to predict prices on "future_data" dataset.

In [15]:
# Create "future_data" dataset
# Future data does not have SOLDPRICE and thus, the feature should be excluded
# NOTE: Future data will have 60 listings - see how we slice the data 
# Export data as "midterm_future_data.csv", storing the dataset in your "data" folder

future_data = condos[9967:]
future_data_features = list(set(future_data.columns) - set(['SOLDPRICE'])) 
future_data = future_data[future_data_features]
future_data.to_csv('static/data/midterm_future_data.csv', index=None)

In [16]:
# Check future_data shape - you shoud see (60, 724)
future_data.shape

(60, 724)

YOUR TURN <br><br>
**Question 4 (1 pt):** <br> *Create "training_data.csv" dataset and store it into your "data" folder.* <br><br>Training data should contain all data except the last 60 obsevrations we have periously exported as "midterm_future_data.csv" data. 

In [17]:
#df.to_csv(static\data\training_data.csv)
training_data = condos[0:9967]
training_data_features = list(set(training_data.columns))
training_data = training_data[training_data_features]
training_data.to_csv('static/data/midterm_training_data.csv', index=None)

In [18]:
# Check the shape of your data - you shoud see (9967, 725)
training_data.shape

(9967, 725)

Before we move on, make sure you have "midterm_future_data.csv" and "midterm_training_data.csv" datasets stored into your "data" folder (they will be together with you "sold_geocoded.csv" dataset):

<img src="static/img/datasets.png" align="left">

## Data Modeling

In this section we are going to train, test, and choose the best performing model. The steps are the following:
    1. Load "midterm_training_data.csv" 
    2. Choose an appropriate feature set
    3. Train/Test data by using 2 models:
        - Random Forest
        - Gradient Boosting
    4. Choose better model and export predictions

#### Load "midterm_training_data.csv" dataset

In [19]:
# Load data
train = pd.read_csv('static/data/midterm_training_data.csv')
train.head()

,ZIP_1029,ZIP_2776,ZIP_2122,ZIP_1929,ZIP_1527,ZIP_1108,ZIP_1719,ZIP_1826,ZIP_2144,ZIP_1760,...,ZIP_1289,ZIP_1262,ZIP_2462,ZIP_2648,ZIP_1098,ZIP_1072,ZIP_1772,ZIP_1522,ZIP_2764,ZIP_2126
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Select features

Exclude 'LISTPRICE','SOLDPRICE', and 'LISTDATE' given that, these are unknown features for any given listing in the future. 'DisplayX'and 'DisplayY' cannot be used as predictors. Usually, they have been used to create distance matrices necessary for estimating the nearest neighbors, which is outside of the scope of this work. 'PHOTOURL' are URL links and 'MLSNUM' is a Database ID. 'REMARKS' are descritpions (we are not doing text analysis here) and thus we are exclusing them as well. We have already extracted condos ('CC') from the 'PROPTYPE' feature.

In [20]:
# Create train_features valiable and store selected features in it
train_features = list(set(train.columns) - set(['PHOTOURL','MLSNUM','SOLDPRICE','REMARKS','LISTDATE',
                                          'LISTPRICE','PROPTYPE', 'DisplayX','DisplayY']))

#### Train/Test data

In [21]:
# Load sklearn library
from sklearn.model_selection import train_test_split

YOUR TURN <br><br>
**Question 5 (1 pt):** <br> *Train - Test - **Split** * <br><br>Use sklearn to split the train dataset into train and test. Specifically, create 'X_train', 'X_test', 'y_train', and 'y_test' datasets. Use 70% for the train and 30% for the test data. Come up with your unique random_state number."  

In [22]:
#sklearn.model_selection.train_test_split(train)
print(train.shape)
print(train[train_features].shape)
y = train['SOLDPRICE']

X_train, X_test, y_train, y_test = train_test_split(train[train_features], y, test_size=0.30, random_state=16)


(9967, 725)
(9967, 716)


In [23]:
# Check the shape of the data
# You should get: 
# X_train.shape = (6976, 716), y_train.shape = (6976,)
# X_test.shape = (2991, 716), y_test.shape = (2991,)

print('X_train.shape = {}, y_train.shape = {}'.format(X_train.shape, y_train.shape))
print('X_test.shape = {}, y_test.shape = {}'.format(X_test.shape, y_test.shape))

X_train.shape = (6976, 716), y_train.shape = (6976,)
X_test.shape = (2991, 716), y_test.shape = (2991,)


### Training Your Models

In [24]:
# Load library
from sklearn.ensemble import RandomForestRegressor

YOUR TURN <br><br>
**Question 6 (1 pt):** <br> *Train data using Random Forest model.*

In [25]:
rf = RandomForestRegressor(n_estimators = 5000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);



In [26]:
y_pred = rf.predict(X_test)

In [27]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 7810.015059969931
Mean Squared Error: 1625685299.507488
Root Mean Squared Error: 40319.78793976337


In [28]:
y_pred[:10]

c = [col for col in X_test.columns if not 'ZIP' in col]

print(X_test[:10][c])
print (y_pred[:10])


        AGE  BEDS  LOTSIZE    PPSF  LISTMONTH  BATHS  GARAGE  SQFT
8031   28.0     2      0.0  240.96          5    1.5     0.0   913
445   143.0     2   9583.0  250.00          9    2.0     1.0  1431
1407  145.0     1    580.0  870.69          2    1.0     0.0   580
6864  113.0     2   3484.0  525.00          3    1.0     0.0   900
7643   32.0     2  16789.0  273.66          5    1.5     0.0   877
6983   13.0     3      0.0  459.65          4    5.5     2.0  3916
8007   18.0     3   1849.0  719.31          5    2.5     2.0  1849
9029   29.0     1    801.0  982.52          7    1.0     1.0   801
9047   34.0     2      0.0  371.31          8    2.0     0.0   983
9178   51.0     1      0.0  235.78          8    1.0     0.0   763
[ 222334.9264  357437.4184  504657.308   478672.5202  239812.3478
 1713090.7072 1353237.81    787939.6628  365818.6406  176993.9042]


In [29]:
# Load library
from sklearn.ensemble import GradientBoostingRegressor

YOUR TURN <br><br>
**Question 7 (1 pt):** <br> *Train data using Gradient Boosting model.*

In [30]:
# gradient boosting ensemble for making predictions for regression
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor

# define the model
model = GradientBoostingRegressor()
# fit the model on the whole dataset
model.fit(X_train, y_train)
# make a single prediction
y_hat = model.predict(X_test)
# summarize prediction


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_hat))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_hat))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_hat)))

Mean Absolute Error: 19164.834141239433
Mean Squared Error: 1473242420.8500504
Root Mean Squared Error: 38382.84018738127


### Predict on Future Data

In [31]:
# Import "future_data" dataset
future_data = pd.read_csv("static/data/midterm_future_data.csv")
future_data.head()  

,ZIP_1029,ZIP_2776,ZIP_2122,ZIP_1929,ZIP_1527,ZIP_1108,ZIP_1719,ZIP_1826,ZIP_2144,ZIP_1760,...,ZIP_1289,ZIP_1262,ZIP_2462,ZIP_2648,ZIP_1098,ZIP_1072,ZIP_1772,ZIP_1522,ZIP_2764,ZIP_2126
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Print future_data columns 
# You can use future_data.columns.values 

future_data.columns

Index(['ZIP_1029', 'ZIP_2776', 'ZIP_2122', 'ZIP_1929', 'ZIP_1527', 'ZIP_1108',
       'ZIP_1719', 'ZIP_1826', 'ZIP_2144', 'ZIP_1760',
       ...
       'ZIP_1289', 'ZIP_1262', 'ZIP_2462', 'ZIP_2648', 'ZIP_1098', 'ZIP_1072',
       'ZIP_1772', 'ZIP_1522', 'ZIP_2764', 'ZIP_2126'],
      dtype='object', length=724)

We want to extract only valid predictors from "future_data" dataset. Note: we don't want to use features such as 'PHOTOURL','MLSNUM' or 'LISTPRICE' as our predictors.

YOUR TURN <br><br>
**Question 8 (1 pt):** <br> Create "predicion_features" variable to store the approprate set of features. <br>Note: The number of features needs to coresspond to the features set used for training. 

In [33]:
# YOUR CODE HERE
prediction_features = list(set(future_data.columns) - set(['PHOTOURL', 'MLSNUM', 'LISTPRICE', 'LISTDATE','REMARKS','DisplayX','DisplayY','PROPTYPE']))



In [34]:
# Create new dataset called "predictions_df"
# The dataset will contain only "predicion_features" set

predictions_df = future_data[prediction_features]

YOUR TURN <br><br>
**Question 9 (1 pt):** <br> Predict prices using your best model. Store predictions in the variable called "predictions". 

In [35]:
# YOUR CODE HERE
for n in X_train.columns:
    if "ZIP" in n:
        continue
    else:
        print(n,X_train[n][0])
print("-----")
for n in predictions_df.columns:
    if "ZIP" in n:
        continue
    else:
        print(n,predictions_df[n][0])
        
predictions = rf.predict(predictions_df)


AGE 31.0
BEDS 3
LOTSIZE 938718.0
PPSF 143.16
LISTMONTH 5
BATHS 2.5
GARAGE 1.0
SQFT 2389
-----
AGE 19.0
BEDS 1
LOTSIZE 418.0
PPSF 1244.02
LISTMONTH 11
BATHS 1.0
GARAGE 0.0
SQFT 418


In [36]:
# Round prediction values 
predictions = predictions.round(2)
predictions

array([ 527242.68,  696756.15,  394522.81, 1630461.24,  324522.82,
        632880.08,  560896.66,  476164.26,  976134.69,  310793.85,
        284751.12,  364768.78, 2150298.9 ,  529493.22,  543321.96,
        149124.6 ,  369408.22,  531858.57,  230621.76,  172797.4 ,
        432713.72,  550262.49,  177314.83,  400487.81,  273108.55,
        514852.1 ,  504890.23,  116590.74,  685290.64,  492180.71,
        140784.14,  549067.82,  582910.54,  239247.23,  451636.34,
        305403.54,   77101.95,  499128.54,  300010.18,  195931.03,
        942585.41,  878977.99,  701919.1 ,  480363.98,  226644.66,
        453313.86,  288017.23,  479004.7 ,  391925.12,  680327.24,
        232082.98,  417055.68,  719706.99,  168145.99,  405422.6 ,
        375582.14,  315378.42,  206150.65,  532724.48,  172027.23])

In [37]:
# Add new column 'predicted_price' to the 'future_data' dataset
# Pass newly predicted values

future_data['predicted_price'] = predictions

In [38]:
# Choose the final set of columns to be use for you final export

final_cols = ['DisplayY','DisplayX','BEDS','SQFT','PHOTOURL','PPSF',
            'BATHS','MLSNUM','LISTPRICE', 'REMARKS', 'predicted_price']

In [39]:
# Create new dataframe for export
# NOTE: This dataframe will be exported to csv file and used in Midterm_Part_II

for_export_df = future_data[final_cols]

In [40]:
# Change column names (small caps)
for_export_df = for_export_df.rename(columns={'REMARKS':'remarks', 'LISTPRICE':'list_price', 'DisplayY':'display_y', 
                                        'DisplayX':'display_x', 'BEDS':'beds', 'SQFT':'sqft', 'PHOTOURL':'photo_url', 
                                        'PPSF':'ppsf', 'BATHS':'baths', 'MLSNUM':'mlsnum'})

In [41]:
# Round lon/lat data
# Parse MLS number to int 
# Note: For MLS number, you may want to use string later, in the Midterm_Part_II 

for_export_df['display_y'] = for_export_df['display_y'].round(5)
for_export_df['display_x'] = for_export_df['display_x'].round(5)
for_export_df['mlsnum'] = for_export_df['mlsnum'].astype(int)

In [42]:
# Final check
for_export_df.head()

,display_y,display_x,beds,sqft,photo_url,ppsf,baths,mlsnum,list_price,remarks,predicted_price
0,42.34879,-71.07807,1,418,http://media.mlspin.com/photo.aspx?mls=72250173,1244.02,1.0,72250173,519000.0,Excellent opportunity to own a studio at Trini...,527242.68
1,42.36682,-71.03763,3,1426,http://media.mlspin.com/photo.aspx?mls=72251181,490.18,2.0,72251181,699000.0,"Newly renovated, 3 bedroom +study, 2 bath feat...",696756.15
2,42.26208,-71.52786,2,2100,http://media.mlspin.com/photo.aspx?mls=72250996,189.05,2.5,72250996,409000.0,Lovely townhome in The Preserve a gated commun...,394522.81
3,42.35796,-71.05054,1,1230,http://media.mlspin.com/photo.aspx?mls=72254711,1341.46,1.5,72254711,1650000.0,Seldom available opportunity to have the covet...,1630461.24
4,42.61405,-71.22847,2,1040,http://media.mlspin.com/photo.aspx?mls=72250876,302.88,2.5,72250876,304900.0,Looking for that well priced Townhouse that is...,324522.82


Your dataset (from above) should look like this (**'predicted_price' figures may be different due to dfferent splits and that is ok! **):<br>
<img src="static/img/final_dataset.png" align="center">

YOUR TURN <br><br>
**Question 10 (1 pt):** <br> Export "for_export_df" dataframe into "midterm.csv" file and store it into your "data" folder. You will model the "condos" database table according to it, so make sure you have completed this step!

In [44]:
# YOUR CODE HERE
for_export_df.to_csv('static/data/midterm.csv')





PermissionError: [Errno 13] Permission denied: 'static/data/midterm.csv'

In [45]:
# Check the dataset shape - you should see (60, 11)
for_export_df.shape

(60, 11)

Your "data" folder should look like this (together with "sold_geocoded.csv"):

<img src="static/img/data2.png" align="left">

Congratulations! You finished the Part I. :)

In [46]:
#Sources: 
#https://stackabuse.com/gradient-boosting-classifiers-in-python-with-scikit-learn/
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html
#https://stackoverflow.com/questions/35261055/method-object-is-not-subscriptable-dont-know-whats-wrong
#https://realpython.com/python-traceback/
#https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/
#https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
#https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html

In [46]:
#Sources:
#https://datatofish.com/create-database-python-using-sqlite3/